## Summarize large documents using LangChain and Gemini

In [4]:
# Installing the necessary libraries

!pip install langchain
!pip install langchain-google-genai
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00


In [2]:
# Accessing the API keys
import os
import getpass

os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [5]:
# Importing the required libraries

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [6]:
# Loading the Data

loader = WebBaseLoader("https://blog.google/technology/ai/google-gemini-ai/#sundar-note")
docs = loader.load()

len(docs)

1

In [7]:
# Splitting the documents into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
data = text_splitter.split_documents(docs)

len(data)

42

In [8]:
# Initialize the Gemini API

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5, top_p = 0.85)

In [26]:
# Creating a map_reduce chain
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose = False)

In [27]:
import textwrap
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Google's Gemini AI model excels in various benchmarks, showcasing multimodal capabilities across
text, code, audio, images, and videos. It prioritizes responsibility and safety through
comprehensive evaluations and external partnerships. Gemini is being integrated into Google products
and will be accessible via APIs for developers and enterprise customers.


In [10]:
# Creating stuff chain

prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""

prompt = PromptTemplate.from_template(prompt_template)

In [31]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [33]:
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="text"
  )

In [34]:
output_summary = stuff_chain.run(docs)

In [39]:
wrapped_text = textwrap.fill(output_summary,
                             width=200,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Google introduces Gemini, its most advanced AI model yet, with state-of-the-art performance across various benchmarks. Gemini is multimodal, flexible, and optimized for different sizes. It excels in
sophisticated reasoning, understanding text, images, audio, and advanced coding. Gemini is designed with responsibility and safety in mind, undergoing comprehensive safety evaluations and
incorporating dedicated safety classifiers. Google is rolling out Gemini Pro in products like Bard and Pixel and making it available via APIs for developers and enterprise customers. Gemini Ultra, the
most capable version, will be released after further safety checks and feedback from select users.
